## This example applies DeepPhasePick on seismic data stored in an unstructured archive directory.

In [9]:
import os
os.chdir('../')
import config, data, model, util                                                

## 1. Configure DPP

In [3]:
# config
util.init_session()
dpp_config = config.Config()
dpp_config.set_trigger(pthres_p=[0.9, 0.001], pthres_s=[0.9, 0.001])
dpp_config.set_picking(op_conds=['1','2','3','4'], tp_th_add=1.5, dt_sp_near=1.5, dt_ps_max=25., dt_sdup_max=2., mcd_iter=10, run_mcd=True)
# dpp_config.set_picking(op_conds=['1','2','3','4'], tp_th_add=1.5, dt_sp_near=1.5, dt_ps_max=25., dt_sdup_max=2., mcd_iter=10, run_mcd=False)
dpp_config.set_data(
    stas=['PB01', 'PB02'],
    net='CX',
    ch='HH',
    archive='sample_data/CX_20140301',
    opath='out_CX_20140301'
)
dpp_config.set_time(
    dt_iter=3600.,
    tstart = "2014-03-01T02:00:00",
    tend = "2014-03-01T03:00:00",        
)

__pycache__/ removed
~/.nv/ not found, continuing...


## 2. Read seismic data into DPP

In [4]:
# data
dpp_data = data.Data()
dpp_data.read_from_directory(dpp_config)
#
# for k in dpp_data.data:
#     print(k, dpp_data.data[k])

reading seismic waveform: sample_data/CX_20140301/CX.PB01..HH.mseed
reading seismic waveform: sample_data/CX_20140301/CX.PB02..HH.mseed
6 Trace(s) in Stream:
CX.PB01..HHZ | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHE | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-03-01T01:59:59.998391Z - 2014-03-01T02:59:59.998391Z | 100.0 Hz, 360001 samples
CX.PB02..HHE | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
#
processing raw stream data...
detrend...
resampling...
merging...
#
time windows (1) for iteration over continuous waveforms:
[UTCDateTime(2014, 3, 1, 2, 0), UTCDateTime(2014, 3, 1, 3, 0)]

slicing...


## 3. Run phase detection and picking

In [5]:
# model
# dpp_model = model.Model(verbose=False)
dpp_model = model.Model(verbose=False, version_pick_P="20201002_2", ntrials_P=17)
#
print(dpp_model.model_detection['best_model'].summary())
print(dpp_model.model_picking_P['best_model'].summary())
print(dpp_model.model_picking_S['best_model'].summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv1d_1 (Separabl (None, 480, 12)           99        
_________________________________________________________________
activation_1 (Activation)    (None, 480, 12)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 480, 12)           48        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 240, 12)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 240, 12)           0         
_________________________________________________________________
separable_conv1d_2 (Separabl (None, 240, 24)           444       
_________________________________________________________________
activation_2 (Activation)    (None, 240, 24)          

In [6]:
# run phase detection
dpp_model.run_detection(dpp_config, dpp_data, save_dets=False, save_data=False)

#
Calculating predictions for stream: CX.PB01..HH?...
720/720 [==============================] - 36s 50ms/step
3 Trace(s) in Stream:
CX.PB01..HHE | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHZ | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 15, s_picks = 12
#
Calculating predictions for stream: CX.PB02..HH?...
strimming stream: 2, 2
720/720 [==============================] - 37s 52ms/step
3 Trace(s) in Stream:
CX.PB02..HHE | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-03-01T01:59:59.998393Z - 2014-03-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-03-01T01:59:59.998391Z - 2014-03-01T02:59:59.998391Z | 100.0 Hz, 360001 samples
p_picks = 10, s_picks = 8


In [7]:
# run phase picking
dpp_model.run_picking(dpp_config, dpp_data, save_plots=True, save_stats=True, save_picks=False)

#
1, 2014-03-01T02:00:00.000000Z, 2014-03-01T03:00:00.000000Z, PB01
triggered picks (P, S): 15, 12
selected picks (P, S): 11, 6
#
P pick: 1/11


100%|██████████| 10/10 [00:02<00:00,  3.60it/s]


3.36 3.26 3.25 3.27 out_CX_20140301/20140301T020000_20140301T030000 PB01
plotting predicted phase P: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB01_P_01.png
plotting predicted phase P: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB01_P_mc_01.png
tpred = 3.260
terr(1 x pb_std) = (-0.010, +0.010)
pick_class = 0
pb, pb_std = (0.512, 0.158)
#
P pick: 2/11


100%|██████████| 10/10 [00:01<00:00,  6.27it/s]


3.36 3.33 3.33 3.35 out_CX_20140301/20140301T020000_20140301T030000 PB01
plotting predicted phase P: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB01_P_02.png
plotting predicted phase P: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB01_P_mc_02.png
tpred = 3.330
terr(1 x pb_std) = (-0.000, +0.020)
pick_class = 0
pb, pb_std = (0.705, 0.200)
#
P pick: 3/11


...

2.4 2.49 2.4 2.57 out_CX_20140301/20140301T020000_20140301T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_E_03.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_N_03.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_E_mc_03.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_N_mc_03.png
tpred = 2.490
terr(1 x pb_std) = (-0.090, +0.080)
pick_class = 1
pb, pb_std = (0.548, 0.196)
#
S pick: 4/5


100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


2.4 2.53 2.52 2.62 out_CX_20140301/20140301T020000_20140301T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_E_04.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_N_04.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_E_mc_04.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_N_mc_04.png
tpred = 2.530
terr(1 x pb_std) = (-0.010, +0.090)
pick_class = 1
pb, pb_std = (0.538, 0.112)
#
S pick: 5/5


100%|██████████| 10/10 [00:05<00:00,  1.71it/s]


2.4 2.46 2.38 2.56 out_CX_20140301/20140301T020000_20140301T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_E_05.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_N_05.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_E_mc_05.png
plotting predicted phase S: out_CX_20140301/20140301T020000_20140301T030000/pick_plots/PB02_S_N_mc_05.png
tpred = 2.460
terr(1 x pb_std) = (-0.080, +0.100)
pick_class = 1
pb, pb_std = (0.548, 0.247)


## 4. Plot predicted phases

In [8]:
# plots
util.plot_predicted_phases(dpp_config, dpp_data, dpp_model)
# util.plot_predicted_phases(dpp_config, dpp_data, dpp_model, plot_probs=['P','S'], shift_probs=True)

creating plots...
1 PB01 Z 2014-03-01T01:59:59.998393Z 2014-03-01T02:59:59.998393Z
1 PB01 E 2014-03-01T01:59:59.998393Z 2014-03-01T02:59:59.998393Z
1 PB02 Z 2014-03-01T01:59:59.998391Z 2014-03-01T02:59:59.998391Z
1 PB02 E 2014-03-01T01:59:59.998393Z 2014-03-01T02:59:59.998393Z
